In [13]:
from data_pipeline.modular_data_extraction import  create_data_packages
from data_pipeline.np_dataset import NpDataset
import input_mapping.models_torch as models_torch
from data_pipeline.image_transforms import get_transforms


from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision import transforms
import numpy as np


In [2]:
labels_to_encode = np.array(["Age-related Macular Degeneration", "Best Disease", "Bietti crystalline dystrophy",
                              "cataract", "Cone Dystrophie or Cone-rod Dystrophie", "Diabetic Retinopathy",
                              "glaucoma", "Maculopathy", "Myopia", "Normal", "Retinitis Pigmentosa", "Stargardt Disease"])

In [3]:
packages, label_standartizer, label_encoder = create_data_packages(labels_to_encode=labels_to_encode)

In [15]:
model_key = 'resnet18'
transform_type = 'ben'
transforms_config = models_torch.model_dict[model_key]['transforms_config']
transform = get_transforms(transform_name = transform_type, transforms_config = transforms_config)

In [16]:
train_datasets, val_datasets, test_datasets = [], [], []
default_file_reader = lambda x: Image.open(x).convert('RGB')
for train, val, test in packages:
    train_datasets.append(NpDataset(file_paths=train.get_data()[:,1], labels=train.get_labels(), file_reader=default_file_reader, transform=transform))
    val_datasets.append(NpDataset(file_paths=val.get_data()[:,1], labels=val.get_labels(), file_reader=default_file_reader, transform=transform))
    test_datasets.append(NpDataset(file_paths=test.get_data()[:,1], labels=test.get_labels(), file_reader=default_file_reader, transform=transform))



In [17]:
train_dataset = ConcatDataset(train_datasets)
val_dataset = ConcatDataset(val_datasets)
test_dataset = ConcatDataset(test_datasets)


In [18]:
#create concatenate all the datasets

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [20]:
# create model
model = models_torch.get_model(model_name=model_key, num_classes=len(labels_to_encode), pretrained=True)
model = model.cuda()
#do some predictions
for i, data in enumerate(test_loader):
    inputs, labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()
    outputs = model(inputs)